# Klasifikace jednotlivých kosatců
Ukázka učení s učitelem pro problematiku klasifikace s využitím Decision Trees a PCA.

In [ ]:
###
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## Příprava dat
Načteme dataset IRIS - velikost kosatců

In [ ]:
df_iris = sns.load_dataset("iris")
df_iris

Vytvoříme sloupec `species_id`, který obsahuje kód kategorií (accessor `cat.codes`)

In [ ]:
df_iris["species"] = df_iris["species"].astype("category")
df_iris["species_id"] = df_iris["species"].cat.codes
df_iris

Vytvoříme data (matice příznaků `X` a vektor labelů `y`)

In [ ]:
ks = ["sepal_length", "sepal_width", "petal_length", "petal_width"] ###
X = df_iris[ks].values
y = df_iris["species_id"].values
X, y

Vytvoříme podmnožinu pro trénování a testování (70:30)

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)
X_test.shape, X_train.shape

## Vytvoření modelu DecisionTreeClassifier

In [ ]:
# Vytvoříme model a rovnou jej otestujeme
model = DecisionTreeClassifier().fit(X_train, y_train)

_  # model.score(X_test, y_test)
_ = _  # (model.predict(X_test) == y_test).mean()

In [ ]:
###
# Vykreslíme rozhodovací strom nového modelu
plt.figure(figsize=(12, 10))
_= plot_tree(model, feature_names=ks, filled=True, class_names=df_iris["species"].cat.categories)

In [ ]:
###
# Podíváme se na kompletní distribuci a korelaci jednotlivých příznaků
sns.pairplot(data=df_iris, vars=ks, hue="species")

## Omezený prostor pro hledání
Z důvodu zrychlení a zjednodušení zkusíme omezit počet příznaků, se kterými se pracuje. V tomto případě to není úplně nutné, ale pokud máme stovky příznaků, nemusí daný algoritmus dobře konvergovat, mohou se vyskytovat korelace a podobně. 

Proto zkusme __omezit počet příznaků na 2__.

In [ ]:
###
# Pro všechny kombinace dvojic zkusíme navrhnout 
from itertools import combinations
for ids in combinations(range(4), 2):
    model = DecisionTreeClassifier().fit(X_train[:, ids], y_train)
    print("score {} = {:.2%}".format([ks[i] for i in ids], model.score(X_test[:, ids], y_test)))


Nejlepší kombinace je použít `petal_length` a `petal_width` (sloupce 2 a 3 z matice X). Zkusme se tedy podívat detailně na fungování.

In [ ]:
###
plt.figure(figsize=(12, 10))
# Vytvoření vektorů x a y (petal_lenght a petal_width)
v_x = X[:, 2]
v_y = X[:, 3]

# Navržení klasifikátoru
m = DecisionTreeClassifier().fit(X_train[:, [2, 3]], y_train)

# Pro vykreslení pozadí uděláme pravidelnou mřížku
xx, yy = np.meshgrid(np.arange(v_x.min(), v_x.max(), 0.002), 
                     np.arange(v_y.min(), v_y.max(), 0.002)
                     )
# kterou transformujeme do matice o rozměrech (-1, 2)
xdata = np.dstack([xx.ravel(), yy.ravel()]).reshape(-1, 2)
# Vypočítáme odezvu našeho modelu
Z = m.predict(xdata)
# a data vykreslíme
plt.contourf(Z.reshape(xx.shape), cmap=plt.cm.tab10, extent=(xx.min(), xx.max(), yy.min(), yy.max()), alpha=0.2)

# vykreslíme všechny body s barvou podle třídy
plt.scatter(v_x, v_y, c=y / 3, cmap=plt.cm.tab10, s=10, clip_on=False)

# nalezeni nesprávnych odpovedi
failed = (m.predict(X[:, [2, 3]]) != y)
plt.scatter(v_x[failed], v_y[failed], ec="tab:red", fc=(1, 1, 0, 0), marker="o", s=30, lw=1)

# formátování grafu
plt.xlabel(ks[2])
plt.ylabel(ks[3])
plt.xlim(v_x.min(), v_x.max())
plt.ylim(v_y.min(), v_y.max())

plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
plot_tree(m, feature_names=ks[2:4], class_names=df_iris["species"].cat.categories, filled=True)
plt.plot()

Jak to ale automatizovat? Prohledávat všechny kombinace může být časově extrémě náročné. Proto můžeme použít __PCA__, které redukuje počet příznaků. Teoreticky se může zlepšit klasifikační přesnost a také 

In [ ]:
from sklearn.pipeline import make_pipeline ###
from sklearn.decomposition import PCA ###
model = make_pipeline(PCA(n_components=2), DecisionTreeClassifier()).fit(X_train, y_train)
print("{:.2%}".format(model.score(X_test, y_test)))

Podíváme se na to, jak vypadají jednotlivé komponenty

In [ ]:
###
model[0].components_ ###
import pandas as pd
pd.DataFrame(model[0].components_, columns=ks, index=["pc1", "pc2"])

In [ ]:
###
plt.figure(figsize=(12, 10))
plot_tree(m, feature_names=["pc1", "pc2"], class_names=df_iris["species"].cat.categories, filled=True)
plt.plot()

Tady jsme dostali podobný výsledek, jako při vlastním výběru feature vektorů. Výhodou PCA je ale, že jsme výsledek dostali najednou, nemuseli jsme zkoumat více kombinací.